In [ ]:
import os
import sys
sys.path.insert(0, "../")
from loadcredentials import *

In [ ]:
dbusr, dbpassword = read_credentials_from_file("","../db_credentials")

## DB Connect to server

In [ ]:
import pymssql
import pandas as  pd

In [ ]:
conn = pymssql.connect(dbaddress, dbusr, dbpassword, dbname,as_dict=True)

In [ ]:
cursor = conn.cursor()

In [ ]:
ids = [int(i) for i in pd.read_excel('extern_ids.xlsx').iloc[:,0].values]

In [ ]:
query = f"""
SELECT [lSchemaId]
      ,[lSequenceNo]
      ,[lPrevSeqNo]
      ,[lNextSeqNo]
      ,[lObjId]
      ,[strLName]
      ,[strLNamePhon]
      ,[lLNameType]
      ,[strGName]
      ,[strGNamePhon]
      ,[lGNamePos]
      ,[strDoB]
      ,[lDoBPos]
      ,[lNumber]
      ,[strNPref]
      ,[strCamp]
      ,[lPersId]
      ,[strSortValue]
      ,[lGapSize]
      ,[strSortValueAlph]
      ,[strSortValuePhon]
      ,[strPrisNo]
      ,[lCountId]
      ,[strNumber]
  FROM [ousarchiv].[ousarchiv].[PersData]
  where lObjId in (
""" + ",".join([str(i) for i in ids]) +");"

In [ ]:
cursor.execute(query)
records = cursor.fetchall()

In [ ]:
data = pd.DataFrame(records)

In [ ]:
data = data[['lObjId', 'strLName', 'strLNamePhon', 'lLNameType', 'strGName', 'strGNamePhon',
       'lGNamePos', 'strDoB', 'lDoBPos', 'lNumber',
       'lPersId',  'lCountId']]

In [ ]:
def deduplicate(person_df):
    #'strLName', 'strLNamePhon', 'lLNameType', 'strGName', 'strGNamePhon', 'lGNamePos', 'strDoB', 'lDoBPos', 'lNumber','lPersId'
    strLName = " ".join(set(person_df['strLName'].values))
    strLNamePhon = " ".join(set(person_df['strLNamePhon'].values))
    strGName = " ".join(set(person_df['strGName'].values))
    strGNamePhon = " ".join(set(person_df['strGNamePhon'].values))
    strDoB = ";".join(list(set(person_df['strDoB'])))
    lNumber = ";".join(list(set(person_df['strDoB'])))
    return [strLName, strLNamePhon, strGName, strGNamePhon, strDoB, lNumber]
    

In [ ]:
deduplicated_data = data.groupby(['lObjId','lCountId']).apply(deduplicate)
idx = deduplicated_data.index
deduplicated_data = pd.DataFrame(list(deduplicated_data.values), columns=["strLName", "strLNamePhon", "strGName", "strGNamePhon", "strDoB", "lNumber"])
deduplicated_data.index = idx

In [ ]:
deduplicated_data

In [ ]:
deduplicated_data.to_excel("ArolsenIDs.xlsx")